<a href="https://colab.research.google.com/github/spencervagg99/ArticleSummarizer/blob/master/exploreMedequip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore Medequip

In [1]:
# Import the required packages
import nltk
import os
import pickle
import re
import snorkel
import string
import sys
import mysql.connector as connector
import pandas as pd
import numpy as np


cwd = os.getcwd()
filename = 'db_keys.py'
path = cwd.split("/");
fullpath = "/".join(path[0:path.index('home')+2]) + "/"
sys.path.insert(1, fullpath)
from db_keys import keys

In [2]:
# Load in the Medequip Gloves
med_df = pd.read_csv('/home/Resources/datasets/medical-search/Gloves/medequip_gloves.tsv', sep='\t')
med_df['item_description_color_raw'] = med_df.apply(lambda x: [], axis=1)
med_df.head()

,Unnamed: 0,ItemID,ManufacturerID,ManufacturerName,ProductID,ProductName,ProductDescription,ManufacturerItemCode,ItemDescription,ItemImageURL,...,HarmonizedTariffCode,HazMatClass,HazMatCode,PharmacyProductType,NationalDrugCode,BrandID,BrandName,FreightClass,GTIN/UPC,item_description_color_raw
0,0,10001118,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785000,"Surgical Gloves, Size 5½, 50 pr/bx, 4 bx/cs (U...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,60.0,NaN,[]
1,1,10001119,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785001,"Surgical Gloves, Size 6, 50 pr/bx, 4 bx/cs (US...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]
2,2,10001121,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785002,"Surgical Gloves, Size 6½, 50 pr/bx, 4 bx/cs (U...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]
3,3,10001123,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785003,"Surgical Gloves, Size 7, 50 pr/bx, 4 bx/cs (US...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]
4,4,10001125,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785004,"Surgical Gloves, Size 7½, 50 pr/bx, 4 bx/cs (U...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]


In [2]:
# Load in the colors dictionary
with open('/home/Resources/datasets/lexica/colors_dict.pkl', 'rb') as f:
  colors = pickle.load(f)

In [4]:
colors['blue']

['blue',
 'light blue',
 'dark blue',
 'yale blue',
 'pigeon blue',
 'sky blue',
 'independence blue',
 'air force blue',
 'baby blue',
 'navy blue',
 'steel blue',
 'carolina blue',
 'turkish blue',
 'maybe blue',
 'cornflower blue',
 'cornflower',
 'cobalt blue',
 'olympic blue',
 'sapphire blue',
 'azure blue',
 'azure',
 'egyptian blue',
 'denim blue',
 'denim',
 'prussian blue',
 'space blue',
 'blurple',
 'blu']

In [5]:
# populate raw colors for medequip
cleanr = re.compile('\(.*?\)|[Gg]loves*|<.*?>.*?<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
for index, row in med_df.iterrows():
    descrip = []
    if pd.isnull(row.ItemDescription):
        continue
    descrip = re.sub(cleanr, '', row.ItemDescription).strip().lower()
    for main_color in colors.keys():
        for shade in colors[main_color]:
            regex = r"\b%s\b" % shade
            found = re.search(regex, descrip, re.IGNORECASE)
            if (found):
                # if found, only record the main color and the index where it was found
                med_df.at[index, 'item_description_color_raw'].append((main_color, found.span()[0]))
med_df.head()

,Unnamed: 0,ItemID,ManufacturerID,ManufacturerName,ProductID,ProductName,ProductDescription,ManufacturerItemCode,ItemDescription,ItemImageURL,...,HarmonizedTariffCode,HazMatClass,HazMatCode,PharmacyProductType,NationalDrugCode,BrandID,BrandName,FreightClass,GTIN/UPC,item_description_color_raw
0,0,10001118,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785000,"Surgical Gloves, Size 5½, 50 pr/bx, 4 bx/cs (U...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,60.0,NaN,[]
1,1,10001119,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785001,"Surgical Gloves, Size 6, 50 pr/bx, 4 bx/cs (US...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]
2,2,10001121,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785002,"Surgical Gloves, Size 6½, 50 pr/bx, 4 bx/cs (U...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]
3,3,10001123,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785003,"Surgical Gloves, Size 7, 50 pr/bx, 4 bx/cs (US...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]
4,4,10001125,35,Ansell,10000167,ANSELL ENCORE® POWDER-FREE STERILE SURGICAL GL...,NaN,5785004,"Surgical Gloves, Size 7½, 50 pr/bx, 4 bx/cs (U...",https://secure.ndc-inc.com/AIMImage/i09229.jpg,...,NaN,NaN,NaN,NaN,NaN,0,NaN,70.0,NaN,[]


In [6]:
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier

#create color mapping to numeric
label_color_mapping = {}
value = 0
# first map all keys in colors to numeric values
for color in colors.keys():
    label_color_mapping[color] = value
    value += 1 
# then add the 'abstain' label
label_color_mapping['abstain'] = -1
ABSTAIN = label_color_mapping['abstain']


def keyword_lookup(x, keywords, label):
    if pd.isnull(x.ItemDescription):
        return ABSTAIN
    cleanr = re.compile('\(.*?\)|[Gg]loves*|<.*?>.*?<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    for word in keywords:
      regex = r"\b%s\b" % word
      desc = re.sub(cleanr, '', x.ItemDescription).strip()
      if (re.search(regex, desc, re.IGNORECASE)):
        # get list of tuples from dataframe and transform them to dict
        # [(violet, 38), (natural, 33)] => {'violet':38, 'natural':33}
        # get the max from the values (i.e. last color mentioned in the description)
        matches = dict(x.item_description_color_raw)
        last_mention = max(matches.values())
        # note: keywords[0] has the main color label
        if matches[keywords[0]] == last_mention:
          return label
    return ABSTAIN


def make_keyword_lf(keywords, label=""):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label),
    )

# dynamically create keyword_color keywords based on lists
for color in label_color_mapping.keys():
  if color != 'abstain':
    keyword_name = "keyword_%s" % (color.lower())
    globals()[keyword_name] = make_keyword_lf(keywords=colors[color], label=label_color_mapping[color])

In [7]:
lfs = [
       keyword_yellow,
       keyword_orange,
       keyword_red,
       keyword_pink,
       keyword_violet,
       keyword_blue,
       keyword_green,
       keyword_brown,
       #keyword_grey,
       keyword_natural,
       keyword_clear,
       keyword_white,
       keyword_black
]

In [8]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=med_df, progress_bar=False)

In [9]:
from snorkel.labeling import LFAnalysis

# Polarity: The set of unique labels this LF outputs (excluding abstains)
# Coverage: The fraction of the dataset the LF labels
# Overlaps: The fraction of the dataset where this LF and at least one other LF label
# Conflicts: The fraction of the dataset where this LF and at least one other LF label and disagree
# Correct: The number of data points this LF labels correctly (if gold labels are provided)
# Incorrect: The number of data points this LF labels incorrectly (if gold labels are provided)
# Empirical Accuracy: The empirical accuracy of this LF (if gold labels are provided)
LFAnalysis(L_train, lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
keyword_yellow,0,[0],0.004575,0.0,0.0
keyword_orange,1,[1],0.014743,0.0,0.0
keyword_red,2,[2],0.001017,0.0,0.0
keyword_pink,3,[3],0.008134,0.0,0.0
keyword_violet,4,[4],0.008134,0.0,0.0
keyword_blue,5,[5],0.167768,0.0,0.0
keyword_green,6,[6],0.043721,0.0,0.0
keyword_brown,7,[7],0.011693,0.0,0.0
keyword_natural,8,[9],0.023894,0.0,0.0
keyword_clear,9,[10],0.004575,0.0,0.0


In [ ]:
cleanr = re.compile('\(.*?\)|[Gg]loves*|<.*?>.*?<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
print(temp[75])
for i in med_df.ItemDescription.values:
    try:
        if not pd.isnull(i):
            print([j.strip().lower() for j in re.sub(cleanr, '', i).split(',') if j != ''])
    except:
        print('Error caused by: ', i)
        break

## Color Labelling Joined Dataframe
We are going to see if the GUDUID and Medequip descriptions both lead to the same results

In [3]:
mapped_df = pd.read_csv('/home/Resources/datasets/medical-search/Gloves/database_matches.tsv', sep='\t')
mapped_df['item_description_color_raw'] = mapped_df.apply(lambda x: [], axis=1)
mapped_df['device_description_color_raw'] = mapped_df.apply(lambda x: [], axis=1)
cleanr = re.compile('\(.*?\)|[Gg]loves*|<.*?>.*?<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
mapped_df.ItemDescription = [re.sub(cleanr, '', str(x)).strip() for x in mapped_df.ItemDescription.values]
mapped_df.head()

,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,device_count,...,HazMatClass,HazMatCode,PharmacyProductType,NationalDrugCode,BrandID,BrandName,FreightClass,GTIN/UPC,item_description_color_raw,device_description_color_raw
0,0,00787081126866,['LZA'],['Polymer Patient Examination Glove'],Hand Armor,GDNPF105XX,GDNPF105XX,90125555,TRANZONIC ACQUISITION CORP.,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
1,1,00787081126859,['LZA'],['Polymer Patient Examination Glove'],Hand Armor,GDNPF105X,GDNPF105X,90125555,TRANZONIC ACQUISITION CORP.,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
2,2,00787081126842,['LZA'],['Polymer Patient Examination Glove'],Hand Armor,GDNPF105L,GDNPF105L,90125555,TRANZONIC ACQUISITION CORP.,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
3,3,00787081126835,['LZA'],['Polymer Patient Examination Glove'],Hand Armor,GDNPF105M,GDNPF105M,90125555,TRANZONIC ACQUISITION CORP.,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
4,4,00787081126828,['LZA'],['Polymer Patient Examination Glove'],Hand Armor,GDNPF105S,GDNPF105S,90125555,TRANZONIC ACQUISITION CORP.,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [4]:
mapped_df = mapped_df[mapped_df.ProductID.notna()].reset_index()
mapped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 52 columns):
index                           548 non-null int64
Unnamed: 0                      548 non-null int64
primary_di                      548 non-null object
product_code                    548 non-null object
product_code_name               548 non-null object
brand_name                      548 non-null object
version_model_number            548 non-null object
catalog_number                  548 non-null object
duns_number                     548 non-null int64
company_name                    548 non-null object
device_count                    548 non-null int64
device_description              548 non-null object
device_sterile                  166 non-null object
ItemID                          548 non-null float64
ManufacturerID                  548 non-null float64
ManufacturerName                548 non-null object
ProductID                       548 non-null float64
ProductName 

In [5]:
# populate raw colors
for index, row in mapped_df.iterrows():
    for main_color in colors.keys():
        for shade in colors[main_color]:
            regex = r"\b%s\b" % shade
            found_gudid = re.search(regex, str(row.device_description), re.IGNORECASE)
            if (found_gudid):
                # if found, only record the main color and the index where it was found
                mapped_df.at[index, 'device_description_color_raw'].append((main_color, found_gudid.span()[0]))
            if row.ItemDescription:
                found_med = re.search(regex, str(row.ItemDescription), re.IGNORECASE)
                if found_med:
                    mapped_df.at[index, 'item_description_color_raw'].append((main_color, found_med.span()[0]))
mapped_df.head()

,index,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,...,HazMatClass,HazMatCode,PharmacyProductType,NationalDrugCode,BrandID,BrandName,FreightClass,GTIN/UPC,item_description_color_raw,device_description_color_raw
0,33,33,00699320452844,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG200BLK,NLG200BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 38)]","[(black, 45)]"
1,34,34,00699320452837,['LZA'],['Polymer patient examination glove'],AMBITEX,NMD200BLK,NMD200BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 39)]","[(black, 45)]"
2,35,35,00699320452820,['LZA'],['Polymer patient examination glove'],AMBITEX,NSM200BLK,NSM200BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 38)]","[(black, 45)]"
3,39,39,00699320422854,['LZA'],['Polymer patient examination glove'],AMBITEX,NXL720BLK,NXL720BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 40)]","[(black, 45)]"
4,40,40,00699320422847,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG720BLK,NLG720BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 38)]","[(black, 45)]"


In [111]:
count_both = 0
count_item = 0
count_device = 0
for i, v in mapped_df.iterrows():
    item = v.item_description_color_raw
    device = v.device_description_color_raw
    if item and device:
        count_both += 1
        if item[0][0] != device[0][0]:
            print(i, item, device)
    elif item and not device:
        count_item += 1
        print('Item: ', i, item)
        print(v.device_description)
        print(v.ItemDescription)
    elif device and not item:
        count_device += 1
        print('Device: ', i, device)
        print(v.device_description)
        print(v.ItemDescription)
print(count_both, count_item, count_device)

Device:  12 [('blue', 42)]
AMBITEX GLV EXAM NON-STERILE NITRILE N400 BLUE POWDER-FREE EXTRA LARGE FINGERTIP TEXTURED STANDARD LENGTH 10/100
, Exam, Nitrile, X-Large, Powder-Free , 100/bx 10 bx/cs
Device:  13 [('blue', 42)]
AMBITEX GLV EXAM NON-STERILE NITRILE N400 BLUE POWDER-FREE LARGE FINGERTIP TEXTURED STANDARD LENGTH 10/100
, Exam, Nitrile, Large, Powder-Free , 100/bx 10 bx/cs
Device:  14 [('blue', 37)]
AMBITEX GLV EXAM NON-STERILE NITRILE BLUE N400 POWDER-FREE MEDIUM FINGERTIP TEXTURED STANDARD LENGTH 10/100
, Exam, Nitrile, Medium, Powder-Free , 100/bx 10 bx/cs
Device:  15 [('blue', 42)]
AMBITEX GLV EXAM NON-STERILE NITRILE N400 BLUE POWDER-FREE SMALL FINGERTIP TEXTURED STANDARD LENGTH 10/100
, Exam, Nitrile, Small, Powder-Free , 100/bx 10 bx/cs
Device:  16 [('blue', 42)]
AMBITEX GLV EXAM NON-STERILE NITRILE N400 BLUE POWDER-FREE EXTRA SMALL FINGERTIP TEXTURED STANDARD LENGTH 10/100
, Exam, Nitrile, X-Small, Powder-Free , 100/bx 10 bx/cs
Device:  22 [('clear', 40)]
AMBITEX GLV EX

In [112]:
print(mapped_df.iloc[269].ItemDescription)
mapped_df.iloc[269]

, Surgical, Latex with Nitrile Coating, Powder-Free , Light Brown, Size 8.5, Sterile, 50/bx, 4 bx/cs


index                                                                        1437
Unnamed: 0                                                                   1437
primary_di                                                         10885380032059
product_code                                                              ['KGO']
product_code_name                                            ["Surgeon's gloves"]
brand_name                                                               PROTEXIS
version_model_number                                                    2D72NS85X
catalog_number                                                          2D72NS85X
duns_number                                                             961027315
company_name                                             Cardinal Health 200, LLC
device_count                                                                    1
device_description              Sterile Latex Powder-Free Surgical Gloveswith ...
device_sterile  

In [ ]:
#create color mapping to numeric
label_color_mapping = {}
value = 0
# first map all keys in colors to numeric values
for color in colors.keys():
    label_color_mapping[color] = value
    value += 1 
# then add the 'abstain' label
label_color_mapping['abstain'] = -1
ABSTAIN = label_color_mapping['abstain']


def keyword_lookup_med(x, keywords, label):
    if pd.isnull(str(x.ItemDescription)):
        return ABSTAIN
    for word in keywords:
      regex = r"\b%s\b" % word
      desc = str(x.ItemDescription)
      if (re.search(regex, desc, re.IGNORECASE)):
        # get list of tuples from dataframe and transform them to dict
        # [(violet, 38), (natural, 33)] => {'violet':38, 'natural':33}
        # get the max from the values (i.e. last color mentioned in the description)
        matches = dict(x.item_description_color_raw)
        last_mention = max(matches.values())
        # note: keywords[0] has the main color label
        if matches[keywords[0]] == last_mention:
          return label
    return ABSTAIN

def keyword_lookup_gudid(x, keywords, label):
    if pd.isnull(x.device_description):
        return ABSTAIN
    for word in keywords:
      regex = r"\b%s\b" % word
      desc = str(x.device_description)
      if (re.search(regex, desc, re.IGNORECASE)):
        # get list of tuples from dataframe and transform them to dict
        # [(violet, 38), (natural, 33)] => {'violet':38, 'natural':33}
        # get the max from the values (i.e. last color mentioned in the description)
        matches = dict(x.device_description_color_raw)
        last_mention = max(matches.values())
        # note: keywords[0] has the main color label
        if matches[keywords[0]] == last_mention:
          return label
    return ABSTAIN

def make_keyword_lf_med(keywords, label=""):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup_med,
        resources=dict(keywords=keywords, label=label),
    )

def make_keyword_lf_gudid(keywords, label=""):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup_gudid,
        resources=dict(keywords=keywords, label=label),
    )

# dynamically create keyword_color keywords based on lists
for color in label_color_mapping.keys():
  if color != 'abstain':
    keyword_name = "keyword_%s" % (color.lower())
    globals()[keyword_name] = make_keyword_lf_med(keywords=colors[color], label=label_color_mapping[color])

## Extracting More Than Colors
Now that we know that the Medequip descriptions can add a lot of value, we will use both GUDID and Medequip to extract other attributes that we want

In [6]:
# Load in Out of Vocab Dictionary
oov = pd.read_csv('/home/Resources/datasets/medical-search/Gloves/norm_map.tsv', sep='\t')
oov = dict(zip(oov['Map from'], oov['Map to']))

In [7]:
def clean_gudid_descriptions(descrip: list, oov: dict) -> list:
    lst = []
    punc = re.sub('[-/.,]', '', string.punctuation) # All punctuation but '-' '.' ',' and '/'
    token_remove = set(string.punctuation) | {'-', '–', 'glove', 'glove-', 'gloves', 'gloves-'}
    for ind, v in enumerate(descrip):
        x = str(v).lower().translate(str.maketrans(punc, ' '*len(punc)))
        x = re.sub(r'\(.*\)', '', x) # Remove annything in parentheses
        x = re.sub(r'([0-9]+)(\s+)((mil))', r'\g<1>\g<3>', x) # Change '3 mil' to 3mil
        x = re.sub(r'([0-9]+)(\s+)(gram|g)', r'\g<1>g', x) # Change 3 g and 3 gram to 3g
        x = re.sub(r'powder\s+free', r'powder-free', x) # Correct powder free to our standardized powder-free
        x = re.sub(r'non\s+sterile', r'non-sterile', x) # Correct non sterile to our standardized non-sterile
        x = re.sub(r'extended\s+cuff', r'extended-cuff', x) # Correct extended cuff to our standardized extended-cuff
        x = re.sub(r'(extra)(\s+)(extra)(\s+)(large|small)', r'\g<1>-\g<3>-\g<5>', x) # Correct extra extra (small|large) to extra-extra-(small|large)
        x = re.sub(r'(extra|x)(\s+)(large|small)', r'\g<1>-\g<3>', x) # Correct (extra|x) (small|large) to extra-(small|large)
        x = [i.strip() for i in nltk.word_tokenize(x)] # Medequip is pretty good about being split by comma
        x = [oov[t] if t in oov else t for t in x] # Replace out of vocab words
        #x = [' '.join(token) for token in x]
        lst.append(' '.join(x))
    return lst

def clean_medequip_descriptions(descrip: list, oov: dict) -> list:
    lst = []
    punc = re.sub('[-/.,]', '', string.punctuation) # All punctuation but '-' '.' ',' and '/'
    token_remove = set(string.punctuation) | {'-', '–', 'glove', 'glove-', 'gloves', 'gloves-'}
    for ind, v in enumerate(descrip):
        x = str(v).lower().translate(str.maketrans(punc, ' '*len(punc)))
        x = re.sub(r'\(.*\)', '', x) # Remove annything in parentheses
        x = re.sub(r'([0-9]+)(\s+)((mil))', r'\g<1>\g<3>', x) # Change '3 mil' to 3mil
        x = re.sub(r'([0-9]+)(\s+)(gram|g)', r'\g<1>g', x) # Change 3 g and 3 gram to 3g
        x = re.sub(r'powder\s+free', r'powder-free', x) # Correct powder free to our standardized powder-free
        x = re.sub(r'non\s+sterile', r'non-sterile', x) # Correct non sterile to our standardized non-sterile
        x = re.sub(r'extended\s+cuff', r'extended-cuff', x) # Correct extended cuff to our standardized extended-cuff
        x = re.sub(r'(extra)(\s+)(extra)(\s+)(large|small)', r'\g<1>-\g<3>-\g<5>', x) # Correct extra extra (small|large) to extra-extra-(small|large)
        x = re.sub(r'(extra|x)(\s+)(large|small)', r'\g<1>-\g<3>', x) # Correct (extra|x) (small|large) to extra-(small|large)
        x = [i.strip() for i in x.split(',') if i != ''] # Medequip is pretty good about being split by comma
        x = [oov[t] if t in oov else t for t in x] # Replace out of vocab words
        #x = [' '.join(token) for token in x]
        lst.append(' , '.join(x))
    return lst

In [8]:
mapped_df.ItemDescription = clean_medequip_descriptions(mapped_df.ItemDescription.values, oov)
mapped_df.device_description = clean_gudid_descriptions(mapped_df.device_description.values, oov)

### Sizes

In [9]:
def extract_sizes(row: pd.Series, df: pd.DataFrame, sizes: list, gudid_name: str = 'size_gudid', med_name: str = 'size_med') -> None:
    regex_digit = r'size[\s]?[0-9]+\.?[0-9]*' # Gets size such as 7.5
    found_gudid = re.findall(regex_digit, str(row.device_description).lower(), re.IGNORECASE)
    found_med = re.findall(regex_digit, str(row.ItemDescription), re.IGNORECASE)
    if found_gudid:
        df.at[i, gudid_name] = found_gudid[0]
    if found_med:
        df.at[index, med_name] = found_med[0]
    if found_med or found_gudid:
        return
    for s in sizes:
        regex = r'\b%s\b' % s # Gets the size 
        found_gudid = re.findall(regex, str(row.device_description).lower(), re.IGNORECASE)
        if found_gudid:
            df.at[i, gudid_name] = found_gudid[0]
        found_med = re.findall(regex, str(row.ItemDescription), re.IGNORECASE)
        if found_med:
            df.at[i, med_name] = found_med[0]

In [10]:
sizes = ['extra-small', 'small', 'medium', 'large', 'extra-large', 'extra-extra-large']
mapped_df['size_gudid'] = ''
mapped_df['size_med'] = ''

In [11]:
for i, v in mapped_df.iterrows():
    extract_sizes(v, mapped_df, sizes, 'size_gudid', 'size_med')
mapped_df.head()

,index,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,...,PharmacyProductType,NationalDrugCode,BrandID,BrandName,FreightClass,GTIN/UPC,item_description_color_raw,device_description_color_raw,size_gudid,size_med
0,33,33,00699320452844,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG200BLK,NLG200BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 38)]","[(black, 45)]",large,large
1,34,34,00699320452837,['LZA'],['Polymer patient examination glove'],AMBITEX,NMD200BLK,NMD200BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 39)]","[(black, 45)]",medium,medium
2,35,35,00699320452820,['LZA'],['Polymer patient examination glove'],AMBITEX,NSM200BLK,NSM200BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 38)]","[(black, 45)]",small,small
3,39,39,00699320422854,['LZA'],['Polymer patient examination glove'],AMBITEX,NXL720BLK,NXL720BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 40)]","[(black, 45)]",extra-large,extra-large
4,40,40,00699320422847,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG720BLK,NLG720BLK,961027315,"Cardinal Health 200, LLC",...,NaN,NaN,0.0,NaN,60.0,NaN,"[(black, 38)]","[(black, 45)]",large,large


In [12]:
mapped_df[(mapped_df.size_gudid != mapped_df.size_med) & (mapped_df.size_gudid) & (mapped_df.size_med)]

,index,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,...,PharmacyProductType,NationalDrugCode,BrandID,BrandName,FreightClass,GTIN/UPC,item_description_color_raw,device_description_color_raw,size_gudid,size_med
241,1331,1331,20680651447962,['LZA'],['Polymer patient examination glove'],Halyard,44796,44796,81057389,"O&M HALYARD, INC.",...,NaN,NaN,0.0,NaN,60.0,NaN,[],"[(green, 12)]",large,extra-large
358,1844,1844,10680651330311,['LZA'],['Polymer patient examination glove'],Halyard,33031,33031,81057389,"O&M HALYARD, INC.",...,NaN,NaN,0.0,NaN,60.0,NaN,[],[],large,extra-large
363,1849,1849,10680651330267,['LZA'],['Polymer patient examination glove'],Halyard,33026,33026,81057389,"O&M HALYARD, INC.",...,NaN,NaN,0.0,NaN,60.0,NaN,[],[],large,extra-large
368,1854,1854,10680651142631,['LZC'],"['Patient examination glove, specialty']",Halyard,14263,14263,81057389,"O&M HALYARD, INC.",...,NaN,NaN,0.0,NaN,60.0,NaN,[],"[(violet, 9)]",large,extra-large


### Type 1 - Sterile vs Non-Sterile and Type 2 - Powder vs Powder-Free

In [26]:
def extract_type(row: pd.Series, df: pd.DataFrame, type_: list, gudid_name: str = 'type1_gudid', med_name: str = 'type1_med') -> None:
    for t in type_:
        regex = r'\b%s\b' % t 
        found_gudid = re.findall(regex, str(row.device_description).lower(), re.IGNORECASE)
        if found_gudid:
            df.at[i, gudid_name] = found_gudid[0]
        found_med = re.findall(regex, str(row.ItemDescription), re.IGNORECASE)
        if found_med:
            df.at[i, med_name] = found_med[0]

In [22]:
type1 = ['sterile', 'non-sterile']
type2 = ['powder', 'powder-free']
mapped_df['type1_gudid'] = ''
mapped_df['type1_med'] = ''
mapped_df['type2_gudid'] = ''
mapped_df['type2_med'] = ''

In [23]:
for i, v in mapped_df.iterrows():
    extract_type(v, mapped_df, type1)
    extract_type(v, mapped_df, type2, 'type2_gudid', 'type2_med')
mapped_df.head()

,index,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,...,FreightClass,GTIN/UPC,item_description_color_raw,device_description_color_raw,size_gudid,size_med,type1_gudid,type1_med,type2_gudid,type2_med
0,33,33,00699320452844,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG200BLK,NLG200BLK,961027315,"Cardinal Health 200, LLC",...,60.0,NaN,"[(black, 38)]","[(black, 45)]",large,large,non-sterile,,powder-free,powder-free
1,34,34,00699320452837,['LZA'],['Polymer patient examination glove'],AMBITEX,NMD200BLK,NMD200BLK,961027315,"Cardinal Health 200, LLC",...,60.0,NaN,"[(black, 39)]","[(black, 45)]",medium,medium,non-sterile,,powder-free,powder-free
2,35,35,00699320452820,['LZA'],['Polymer patient examination glove'],AMBITEX,NSM200BLK,NSM200BLK,961027315,"Cardinal Health 200, LLC",...,60.0,NaN,"[(black, 38)]","[(black, 45)]",small,small,non-sterile,,powder-free,powder-free
3,39,39,00699320422854,['LZA'],['Polymer patient examination glove'],AMBITEX,NXL720BLK,NXL720BLK,961027315,"Cardinal Health 200, LLC",...,60.0,NaN,"[(black, 40)]","[(black, 45)]",extra-large,extra-large,non-sterile,,powder-free,powder-free
4,40,40,00699320422847,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG720BLK,NLG720BLK,961027315,"Cardinal Health 200, LLC",...,60.0,NaN,"[(black, 38)]","[(black, 45)]",large,large,non-sterile,,powder-free,powder-free


In [24]:
mapped_df[(mapped_df.type2_gudid != '') & (mapped_df.type2_med != '') & (mapped_df.type2_gudid != mapped_df.type2_med)]

,index,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,...,FreightClass,GTIN/UPC,item_description_color_raw,device_description_color_raw,size_gudid,size_med,type1_gudid,type1_med,type2_gudid,type2_med


### Packaging

In [53]:
def extract_packaging(row: pd.Series, df: pd.DataFrame, gudid_box: str = 'box_gudid', med_box: str = 'box_med',
                      gudid_case = 'case_gudid', med_case = 'case_med') -> None:
    regex_box = r'[0-9]+\.?[0-9]*[\s]*(?:pr|per)?\/(?:box|bx|bag|bg)'
    regex_case = r'[0-9]+\.?[0-9]*[\s]*(?:box|bx|bag|bg)?\/(?:cs|case)'
    regex_case_box = r'[0-9]+\/[0-9]+'
    regex_int = r'[0-9]+\.?[0-9]*'


    found_gudid_box = re.findall(regex_box, str(row.device_description).lower(), re.IGNORECASE)
    found_med_box = re.findall(regex_box, str(row.ItemDescription), re.IGNORECASE)

    found_gudid_case = re.findall(regex_case, str(row.device_description).lower(), re.IGNORECASE)
    found_med_case = re.findall(regex_case, str(row.ItemDescription), re.IGNORECASE)

    found_gudid_case_box = re.findall(regex_case_box, str(row.device_description).lower(), re.IGNORECASE)
    found_med_case_box = re.findall(regex_case_box, str(row.ItemDescription), re.IGNORECASE)


    if found_gudid_box:
        df.at[i, gudid_box] = re.findall(regex_int, found_gudid_box[0])[0]
    if found_med_box:
        df.at[i, med_box] = re.findall(regex_int, found_med_box[0])[0]
    if found_gudid_case:
        df.at[i, gudid_case] = re.findall(regex_int, found_gudid_case[0])[0] 
    if found_med_case:
        df.at[i, med_case] = re.findall(regex_int, found_med_case[0])[0]
    if not (found_gudid_box or found_gudid_case) and found_gudid_case_box:
        case_box = found_gudid_case_box[0].split('/') 
        df.at[i, gudid_case] = case_box[0]
        df.at[i, gudid_box] = case_box[1]
    if not (found_med_box or found_med_case) and found_med_case_box:
        case_box = found_gudid_case_box[0].split('/')
        df.at[i, med_case] =  case_box[0]
        df.at[i, med_box] =  case_box[1]
    return

In [55]:
mapped_df['box_gudid'] = ''
mapped_df['box_med'] = ''
mapped_df['case_gudid'] = ''
mapped_df['case_med'] = ''

for i, v in mapped_df.iterrows():
    extract_packaging(v, mapped_df)
mapped_df.head()

,index,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,...,size_gudid,size_med,type1_gudid,type1_med,type2_gudid,type2_med,box_gudid,box_med,case_gudid,case_med
0,33,33,00699320452844,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG200BLK,NLG200BLK,961027315,"Cardinal Health 200, LLC",...,large,large,non-sterile,,powder-free,powder-free,100,100,10,10
1,34,34,00699320452837,['LZA'],['Polymer patient examination glove'],AMBITEX,NMD200BLK,NMD200BLK,961027315,"Cardinal Health 200, LLC",...,medium,medium,non-sterile,,powder-free,powder-free,100,100,10,10
2,35,35,00699320452820,['LZA'],['Polymer patient examination glove'],AMBITEX,NSM200BLK,NSM200BLK,961027315,"Cardinal Health 200, LLC",...,small,small,non-sterile,,powder-free,powder-free,100,100,10,10
3,39,39,00699320422854,['LZA'],['Polymer patient examination glove'],AMBITEX,NXL720BLK,NXL720BLK,961027315,"Cardinal Health 200, LLC",...,extra-large,extra-large,non-sterile,,powder-free,powder-free,100,100,10,10
4,40,40,00699320422847,['LZA'],['Polymer patient examination glove'],AMBITEX,NLG720BLK,NLG720BLK,961027315,"Cardinal Health 200, LLC",...,large,large,non-sterile,,powder-free,powder-free,100,100,10,10


In [57]:
mapped_df[(mapped_df.case_gudid != '') & (mapped_df.case_med != '') & (mapped_df.case_gudid != mapped_df.case_med)]

,index,Unnamed: 0,primary_di,product_code,product_code_name,brand_name,version_model_number,catalog_number,duns_number,company_name,...,size_gudid,size_med,type1_gudid,type1_med,type2_gudid,type2_med,box_gudid,box_med,case_gudid,case_med


### Use

### Material

### Manufacturer

### Brand

### Glove Thickness

In [60]:
def extract_thickness(row: pd.Series, df: pd.DataFrame, gudid_box: str = 'thickness_gudid', med_box: str = 'thickness_med') -> None:
    

index                                                                          33
Unnamed: 0                                                                     33
primary_di                                                         00699320452844
product_code                                                              ['LZA']
product_code_name                           ['Polymer patient examination glove']
brand_name                                                                AMBITEX
version_model_number                                                    NLG200BLK
catalog_number                                                          NLG200BLK
duns_number                                                             961027315
company_name                                             Cardinal Health 200, LLC
device_count                                                                  100
device_description              ambitex glove exam non-sterile nitrile n200blk...
device_sterile  